In [ ]:
import warnings
warnings.filterwarnings('ignore')

from datascience import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('fivethirtyeight')

# COVID-19 Maps

In this report we introduce maps! Some data has geospatial features such as latitude and longitude, giving us the opportunity to understand how instances of our data are spread across different locations. 

<img src="longitude-and-latitude.png">

## The Data Science Life Cycle - Table of Contents

<a href='#section 0'>Background Knowledge: Spread of Disease</a>

<a href='#subsection 1a'>Formulating a question or problem</a> 

<a href='#subsection 1b'>Acquiring and preparing data</a>

<a href='#subsection 1c'>Conducting exploratory data analysis</a>

<a href='#subsection 1d'>Using prediction and inference to draw conclusions</a>
<br><br>

## Background<a id='section 0'></a>

In March 2020, our lives were turned upside down as the COVID-19 virus spread throughout the United States.  The Centers for Disease Control (CDC) collects data to help health scientists better understand how disease spreads.

Making comparisons between counties and states can us understand how rapidly a virus spreads, the impact of restrictions on public gatherings on the spread of a virus, and measure the changes in fatality as the medical profession learns how to treat the virus and as people get vaccinated. 

A helpful tool in making sense of COVID-19 are maps, usually used to display dot maps that represent total cases, total deaths, total vaccines administered, and more. John's Hopkin's [COVID-19 Map](https://coronavirus.jhu.edu/map.html) provides a great example.

## Formulating a question or problem <a id='subsection 1a'></a>

It is important to ask questions that will be informative and that will avoid misleading results. There are many different questions we could ask about COVID-19, for example, many researchers use data to predict the outcomes based on intervention techniques such as social distancing. 

<div class="alert alert-info">
<b>Question:</b> Take some time to formulate questions you have about this pandemic and the data you would need to answer the questions. 
   </div>

**Your questions:** 

**Data you would need:** 

**Article:**

## Acquiring and preparing data <a id='subsection 1b'></a>

You will be looking at data from the COVID-19 Data Repository at Johns Hopkins University. You can find the raw data [here](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series). 

You will be investigating the total/cumulative number of cases, new cases, and fatalities on a month-to-month basis for counties accross the US, from March 2020 to May 2021.

The following table, `covid_data`, contains the data collected for each month from March 2020 through May 2021 for every county in the United States.

In [ ]:
covid_data = Table().read_table("covid_data/covid_timeseries.csv")

Here are some of the important fields in our data set that you will focus on:

|Variable Name   | Description |
|:---|:---|
|Admin2 | County name |
|Province_State | State name |
|Latitude | Measurement of location north or south of the equator | 
|Longitude | Measurement of location east or west of the prime meridian |
|month| Reporting month represented as the last day of the month, e.g., 3.31.20 |
|total_cases | Cumulative number of COVID cases |
|month_cases| New cases reported in the month |
|total_fatalities | Cumulative number of fatal COVID cases |
|month_fatalities| New fatal cases reported in the month |
|Population | Population in the county |

Let's take a look at the data.

In [ ]:
# Run this cell show the first ten rows of the data
covid_data.show(10)

We are primarily interested in the COVID cases in the United States, so let's select the rows that correspond to the United States. The column <b>iso3</b> will help us select the rows.

Find the United States' country code [here](https://unstats.un.org/unsd/tradekb/knowledgebase/country-code).

In [ ]:
covid_us = covid_data.where('iso3', are.equal_to("..."))

In [ ]:
covid_us.show(10)

For our purposes, we will not be using the columns: "iso3", "Country_Region", "Combined_Key".

We will keep the column FIPS because it uniquely identifies a county. For example, Montana and Wyoming both have a county called "Big Horn".

In [ ]:
cols_to_drop = make_array("...", "...", "...")

covid_us = covid_us.drop(cols_to_drop)

In [ ]:
covid_us.show(10)

Let's give the remaining columns simpler, more meaningful names.

In [ ]:
old_names = make_array('Admin2', 'Province_State', 'month', "Lat", "Long_", "cases", "cases_new", "fatalities", "fatalities_new")
new_names = make_array('County', 'State', 'Date', "Latitude", "Longitude", "Total_Cases", "New_Cases", "Total_Fatalities", "New_Fatalities")

In [ ]:
covid_us = covid_us.relabel(old_names, new_names)

In [ ]:
covid_us.show(10)

One additional change we will execute is to format the date in our dataset. This will allow us to plot specific columns in our data such as New_Cases or New_Fatalities, and allow us to see how these change throughout time. Simply run the cell below, which correctly formats the date in our dataset.

In [ ]:
# Converting date into datetime object
covid_us_pd = covid_us.to_df()
date = pd.to_datetime(covid_us_pd.Date, format='%m/%y')
covid_us['Date'] = date.dt.strftime('%m/%Y')
covid_us.show(10)

### Cases per 100,000 people

There is more than one way to measure the severity of the pandemic. Rather than looking at pure counts, we may want to adjust it according to how many people are in the county. For example, a county with 6,000 people, half of whom are sick, would have 3,000 infected people. Compared to Los Angeles county, this is not alot of cases. However, it is a lot if we think about it in terms of percentages. For this reason, we also want to compare the rates. We could calculate the percentage of cases in the population:

$$100 * cases/population$$

The percentage represents the average number of cases per 100 people. When percentages are small, we often use rates per 10,000 or 100,000 people, i.e.,

$$100000 * cases/population$$

Let's calculate this statistic for our entire dataset by adding a new column entitled cases_per_100k.
As a first step, we drop the counties that don't have a value for population. If you want, you can dig deeper and see which counties these are. It's just a hand full.

In [ ]:
covid_us = covid_us.where("...", are.not_equal_to(0))

<div class="alert alert-info">
<b>Question:</b> Add a column called "Cases_Per100k" that has the number of cases in a county divided by the population of the county.
   </div>

In [ ]:
#What columns should be in the numerator or the denominator 
cases_per100k_array = 100000 * covid_us.column('...') / covid_us.column('...')

#Create a new column called CASES_PER100K in our new table
covid_us = covid_us.with_columns('...', cases_per100k_array)

In [ ]:
covid_us.show(10)

<div class="alert alert-info">
<b>Question:</b> Add a column called "New_Cases_Per100k" that has the new number of cases in a county divided by the population of the county.
   </div>

In [ ]:
#What columns should be in the numerator or the denominator 
new_cases_per100k_array = 100000 * covid_us.column('...') / covid_us.column('...')

#Create a new column called CASES_PER100K in our new table
covid_us = covid_us.with_columns('...', cases_per100k_array)

In [ ]:
covid_us.show(10)

## Conducting exploratory data analysis <a id='subsection 1c'></a>

Often when we begin our explorations, we first narrow down the data to explore. For example, we might choose a particular month to examine, or a particular state, or both. To get us started, let's narrow our exploartions to the first month, March 2020.

In [ ]:
march_2020 = covid_us.where("Date", are.equal_to("03/2020"))
march_2020.show(10)

We will be using <b>dot maps</b> and <b>size maps</b> as a visualization tool to help us understand what the data is telling us. 

### Dot map

Dot maps are a simple map with a dot at each (latidude, longitude) pair from our data. 

The next cell creates a function called <b>dot_map</b> which we will use to create a dot map. 

In [ ]:
def dot_map(tbl):
    """Create a map with dots to represent a unique location.
    
    Parameters:
        tbl (datascience.Table): The Table containing the data needed to plot our map. Note the table
        must have a "Latitude" and "Longitude" column for this function to work.
    Returns:
        (datascience.Map): A map with a dot at each unique (lat, long) pair.
    """
    reduced = tbl.select("Latitude", "Longitude")
    return Circle.map_table(reduced, area=10, fill_opacity=1)

<div class="alert alert-danger" role="alert">
    <b>Example:</b> Let's start with a dot map that displays all of our counties. To do so, we can pass in our table <code>march_2020</code> as an argument to <b>dot_map</b>.
</div>

In [ ]:
dot_map(march_2020)

<div class="alert alert-info">
<b>Question:</b> What inference can we draw from this map? Take a look at the spread of counties.
   </div>

*Insert answer here*

<div class="alert alert-info">
    <b>Question:</b> Assign <code>more_than_100</code> to a table with all counties with more than 100 total cases in March 2020. Pass it in as an argument to <b>dot_map</b>.
   </div>

In [ ]:
more_than_100 = march_2020.where("...", are.above(...))

In [ ]:
dot_map(more_than_100)

<div class="alert alert-info">
    <b>Question:</b> How does the map of counties with more than 100 cases in March 2020 compare to our original size map of all counties in March 2020? What inference can we draw from their differences?
   </div>

*Insert answer here*

<div class="alert alert-info">
    <b>Question:</b> Assign <code>more_than_per100k</code> to a table with all counties with more than 100 cases per 100k in March 2020. Pass it in as an argument to <b>dot_map</b>.
   </div>

In [ ]:
more_than_per100k = march_2020.where("...", are.above(...))

In [ ]:
dot_map(more_than_per100k)

<div class="alert alert-info">
    <b>Question:</b> How does the map of counties with more than 100 cases per 100k in March 2020 compare to our our previous and original size maps? What inference can we draw from their differences? 
   </div>

*Insert answer here*

### Size map

Size maps are detail-oriented maps, using color and size data to add more visual information to our map.

The next cell creates a function called <b>size_map</b> which we will use to create a size map.

In [ ]:
def size_map(tbl):
    """Plots a geographical map where each dot represents a coordinate pair, scaled by a given column.
    
    Parameters:
        tbl: The input Table containing the following arguments, in order:
            Col 0: latitude
            Col 1: longitude
            Col 2: type of location
            Col 3: color (MUST be labeled "colors")
            Col 4: area (MUST be labeled "areas")
    Returns:
        (datascience.Map): A map with a dot at each (lat, long),
                        colored according to Col 3,area as in Col 4.
    """
    return Circle.map_table(tbl, fill_opacity=0.7)

Compared to our function <b>dot_map</b>, this requires a table of a specific format for the table:

| Latitude | Longitude | type | colors | areas
|:---|:---|:---|:---|:---
|...|...|...|...|...

The next two cells create functions <b>get_colors_from_column</b> and <b>get_areas_from_column</b> which will help us create Col 3: colors and Col 4: areas! 

Don't worry about the code. We'll explain how to use them in the example.

In [ ]:
# Col 4: size

def get_areas_from_column(tbl, label):
    """Gets the array values corresponding to the column label in the input table."""
    areas = tbl.column(label)
    areas[areas == 0] = np.nan
    return areas

In [ ]:
# Col 3: color

def get_colors_from_column(tbl, col, include_outliers=False):
    """Assigns each row of the input table to a color based on the value of its percentage column."""
    vmin = min(tbl.column(col))
    vmax = max(tbl.column(col))

    if include_outliers:
        outlier_min_bound = vmin
        outlier_max_bound = vmax
    else:
        q1 = np.percentile(tbl.column(col), 25)
        q3 = np.percentile(tbl.column(col), 75)
        IQR = q3 - q1
        outlier_min_bound = max(vmin, q1 - 1.5 * IQR)
        outlier_max_bound = min(vmax, q3 + 1.5 * IQR)
        
    colorbar_scale = list(np.linspace(outlier_min_bound, outlier_max_bound, 10))
    scale_colors = ['#006100', '#3c8000', '#6ba100', '#a3c400', '#dfeb00', '#ffea00', '#ffbb00', '#ff9100', '#ff6200', '#ff2200']
    
    def assign_color(colors, cutoffs, datapoint):
        """Assigns a color to the input percent based on the data's distribution."""
        for i, cutoff in enumerate(cutoffs):
            if cutoff >= datapoint:
                return colors[i - 1] if i > 0 else colors[0]
        return colors[-1]
    
    colors = [""] * tbl.num_rows
    for i, datapoint in enumerate(tbl.column(col)): 
        colors[i] = assign_color(scale_colors, colorbar_scale, datapoint)
        
    return colors

<div class="alert alert-danger" role="alert">
    <b>Example:</b> Let's start with a size map that displays cases per 100k using the same table <code>march_2020</code>. To do so, we will:
    <ol>
        <li>Pass in our table and column data we wish to work with as our arguments to the function <b>get_colors_from_column</b>. It will return an array with strings that represent colors in hexadecimal format. Larger values will result in green-yellow-orange-red shades in the map.</li>
        <li>Pass in our table and column data we wish to work with as our arguments to the function <b>get_areas_from_columns</b>. It will return an array just like .column does. Larger values will result in larger circles by area in the map.
        <li>Create a new table selecting "Latitude" and "Longitude", then adding in the columns "type", "colors", and "areas". 
    </ol>
</div>

In [ ]:
# Step 1: Use function get_colors_from_column (arguments: march_2020, "Cases_Per100k")
example_colors = get_colors_from_column(march_2020, "Cases_Per100k")

# Step 2: Use function get_areas_from_column (arguments: march_2020, "Cases_Per100k")
example_areas = get_areas_from_column(march_2020, "Cases_Per100k") 

# Step 3: Create a new table with columns "Latitude", "Longitude", "type", "colors", and "areas"
size_per100k = march_2020.select("Latitude", "Longitude").with_columns("type", "Cases_Per100k",
                                                                       "colors", example_colors,
                                                                       "areas", example_areas)

In [ ]:
size_map(size_per100k)

Explore the map! Pan and zoom as you see fit. 

<div class="alert alert-info">
    <b>Question:</b> Compare this first size map to the last dot map. What stands out? 
   </div>

*Insert answere here*

<b>During the holiday season, new cases were reaching all-time highs for numerous reasons, including travel, gatherings, events, and more. 

<div class="alert alert-info">
    <b>Question:</b> Why would January 2021 be a critical month to visualize? Additionally, what is an important consideration to this discussion? Hint: travel
   </div>

*Insert answer here*

<div class="alert alert-info">
    <b>Question:</b> Create a size map that displays new cases per 100k using a new table <code>jan_2021</code>.
   </div>

In [ ]:
jan_2021 = covid_us.where("...", are.equal_to("..."))
jan_2021.show(10)

In [ ]:
q1_colors = get_colors_from_column(jan_2021, "...") 
q1_areas = get_areas_from_column(jan_2021, "...") * 0.5 # Reduce area size by 50%

q1_size_per100k = jan_2021.select("Latitude", "Longitude").with_columns("type", "...",
                                                                          "colors", q1_colors,
                                                                          "areas", q1_areas)

In [ ]:
size_map(q1_size_per100k)

<i>Note: The area sizes for all dots are reduced by 50%</i>

<div class="alert alert-info">
    <b>Question:</b> What stands out from this map?
   </div>

*Insert answer here*

<b> Our data ends at May 2021, so let's take a look at a size map that displays total cases per 100k during that month.</b>

<div class="alert alert-info">
    <b>Question:</b> Create a size map that displays cases per 100k using a new table <code>may_2021</code>.
   </div>

In [ ]:
may_2021 = covid_us.where("...", are.equal_to("..."))
may_2021.show(10)

In [ ]:
q3_colors = get_colors_from_column(..., "...")
q3_areas = get_areas_from_column(..., "...") * 0.10 # Reduce area size by 90%

q3_size_per100k = may_2021.select("Latitude", "Longitude").with_columns("type", "...",
                                                                          "colors", q3_colors,
                                                                          "areas", q3_areas)

In [ ]:
size_map(q3_size_per100k)

<i>Note: The area sizes for all dots are reduced by 90%</i>

<div class="alert alert-info">
    <b>Question:</b> What's the difference between the size map using the table <code>march_2020</code> compared to <code>may_2021</code>? What do the differences tell you? Zoom in and focus on counties you are familiar with. Does this match with your recent knowledge on COVID-19 cases?
   </div>

*Insert answer here*

## Using prediction and inference to draw conclusions <a id='subsection 1d'></a>

<div class="alert alert-info"> 
<b>Question:</b> After seeing these map visualizations, tell us something interesting about this data. What detail were you able to uncover?

</div>

*Insert answer here*

<div class="alert alert-info"> 
<b>Question:</b> What should we consider when looking at this data through maps? What's important to get an accurate read of COVID-19 cases in each county and date? 

</div>

*Insert answer here*

<div class="alert alert-success" role="alert">
  <h2 class="alert-heading">Well done!</h2>
    <p>In this report you used real-world data from the COVID-19 Data Repository at Johns Hopkins University to draw maps that give you more insight on the progression of COVID-19 cases across United States counties. 
    <hr>
    <p> Notebook created for Berkeley Unboxing Data Science 2021 
    <p> Adapted from Project: COVID-19 by Carlos Ortiz with support from Ani Adhikari, Deb Nolan, and Will Furtado
</div>